# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [ ]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain


/bin/bash: line 1: pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain: command not found


In [1]:
!pip install langchain langchain_community unstructured sentence_transformers tiktoken chromadb langchain_chroma langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached n

In [2]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 7.50MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [3]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


In [4]:
!pip install langchain


In [6]:
import pandas as pd
import markdown
import os

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_community.embeddings.sentence_transformer import  SentenceTransformerEmbeddings





# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [7]:
df = pd.read_csv('/content/Dataset.csv')
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [8]:
df.rename(columns={'الغرامة': 'infringement',"المخالفة":"fine"}, inplace=True)
df.head()

,fine,infringement
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [9]:
dirc = 'data/infringement - fine'
os.makedirs(dirc, exist_ok=True)

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [10]:
for i in range(0, len(df)):

    infr = df['infringement'].iloc[i]
    fine = df['fine'].iloc[i]

    markcontent = f"# {infr}\n\n"
    markcontent += f"{fine}\n\n"

    with open(f'{dirc}/{i}.md', 'w', encoding='utf-8') as file:
        file.write(markcontent)

In [11]:
markdown_texts = []
for filename in os.listdir(dirc):
  if filename.endswith(".md"):
    with open(os.path.join(dirc, filename), 'r', encoding='utf-8') as file:
      markcontent = file.read()
      html_content = markdown.markdown(markcontent)
      markdown_texts.append(html_content)

In [13]:
text_spl = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
doc= text_spl.create_documents(markdown_texts)

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [14]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(doc, embedding_function, persist_directory="./chroma_db")

<ipython-input-14-2e7a6cf49366>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://h

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [15]:
pre = '/content/chroma_db'
embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
persist_di= "./chroma_db"
db = Chroma(persist_directory=persist_di, embedding_function=embedding)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [16]:
#def query_chroma_db(query, db, top_k=5):
   # docs = db.similarity_search(query)
   # results = [doc.page_content for doc in docs]
   # return results

In [17]:
#query_chroma_db(" ?ماهي المخالفات ", db)

NameError: name 'query_chroma_db' is not defined

In [18]:
PROMPT_TEMPLATE="""
Question: {question}
Your answer:

Answer the question based only on the following context:
Context: {context}

"""

prompt_template = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)

In [19]:
groq_api_key = "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"
llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-8b-8192")

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [20]:
MODEL = LLMChain(llm=llm,
                 prompt=prompt_template,
                 verbose=True)

<ipython-input-20-a2932d0d6a9b>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  MODEL = LLMChain(llm=llm,


## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [21]:
def query_rag(query: str):
    similarity_search_results = db.similarity_search_with_score(query, k=4)
    context_text = "\n\n".join([doc.page_content for doc, _score in similarity_search_results])

    rag_response = MODEL.invoke({"question": query,"context": context_text})

    return rag_response

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [22]:
response = query_rag("ماهي الغرامة على القيادة بدون رخصة؟")
response

Prompt after formatting:

Question: ماهي الغرامة على القيادة بدون رخصة؟
Your answer:

Answer the question based only on the following context:
Context: <h1>الغرامة المالية 3000 - 6000 ريال</h1>
<p>إجراء سباق للمركبات على الطرق العامة، أو السير في مواكب دون الحصول على تصريح.</p>

<h1>الغرامة المالية 150 - 300 ريال</h1>
<p>الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً.</p>

<h1>الغرامة المالية 1000 - 2000 ريال</h1>
<p>نقل عدد من الركاب يزيد على العدد المحدد في رخصة السير.</p>

<h1>الغرامة المالية 500 - 900 ريال</h1>
<p>القيادة على الطريق بمركبات مخصصة للاقتناء.</p>



> Finished chain.


{'question': 'ماهي الغرامة على القيادة بدون رخصة؟',
 'context': '<h1>الغرامة المالية 3000 - 6000 ريال</h1>\n<p>إجراء سباق للمركبات على الطرق العامة، أو السير في مواكب دون الحصول على تصريح.</p>\n\n<h1>الغرامة المالية 150 - 300 ريال</h1>\n<p>الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً.</p>\n\n<h1>الغرامة المالية 1000 - 2000 ريال</h1>\n<p>نقل عدد من الركاب يزيد على العدد المحدد في رخصة السير.</p>\n\n<h1>الغرامة المالية 500 - 900 ريال</h1>\n<p>القيادة على الطريق بمركبات مخصصة للاقتناء.</p>',
 'text': 'Based on the provided context, the fine for driving without a license is 3000 - 6000 ريال.'}

In [23]:
response2= query_rag("ماهي المخالفات؟ ")
response2

Prompt after formatting:

Question: ماهي المخالفات؟ 
Your answer:

Answer the question based only on the following context:
Context: <h1>الغرامة المالية 150 - 300 ريال</h1>
<p>مخالفة تنظيمات السير على الطرق.</p>

<h1>الغرامة المالية 3000 - 6000 ريال</h1>
<p>طمس لوحات المركبة.</p>

<h1>الغرامة المالية 150 - 300 ريال</h1>
<p>التجمهر في موقع الحادث.</p>

<h1>الغرامة المالية 150 - 300 ريال</h1>
<p>مخالفة قواعد استعمال أنوار التلاقي.</p>



> Finished chain.


{'question': 'ماهي المخالفات؟ ',
 'context': '<h1>الغرامة المالية 150 - 300 ريال</h1>\n<p>مخالفة تنظيمات السير على الطرق.</p>\n\n<h1>الغرامة المالية 3000 - 6000 ريال</h1>\n<p>طمس لوحات المركبة.</p>\n\n<h1>الغرامة المالية 150 - 300 ريال</h1>\n<p>التجمهر في موقع الحادث.</p>\n\n<h1>الغرامة المالية 150 - 300 ريال</h1>\n<p>مخالفة قواعد استعمال أنوار التلاقي.</p>',
 'text': 'Based on the context, the violations mentioned are:\n\n1. مخالفة تنظيمات السير على الطرق (Violating traffic regulations)\n2. طمس لوحات المركبة (Tampering with vehicle plates)\n3. التجمهر في موقع الحادث (Gathering at the scene of an accident)\n4. مخالفة قواعد استعمال أنوار التلاقي (Violating rules for using headlights)\n\nThese are the violations mentioned in the context, with corresponding financial penalties ranging from 150-300 Riyals to 3000-6000 Riyals.'}

In [24]:
response3= query_rag("ماهي عقوبة عدم الوقوف وقوفا تاما عند الاشارة ؟")
response3

Prompt after formatting:

Question: ماهي عقوبة عدم الوقوف وقوفا تاما عند الاشارة ؟
Your answer:

Answer the question based only on the following context:
Context: <h1>الغرامة المالية 5000 - 10000 ريال</h1>
<p>قيادة المركبة تحت تأثير مسكر أو مخدر، أو عقاقير محذر من القيادة تحت تأثيرها.</p>

<h1>الغرامة المالية 150 - 300 ريال</h1>
<p>عدم ترك السائق مسافة كافية بين مركبته والمركبة التي أمامه، تسمح له بتفادي وقوع حادث.</p>

<h1>الغرامة المالية 3000 - 6000 ريال</h1>
<p>تجاوز حافلات النقل المدرسي عند توقفها للتحميل أو التنزيل.</p>

<h1>الغرامة المالية 3000 - 6000 ريال</h1>
<p>تركيب تجهيزات في المركبة كتلك الخاصة بالمركبات الرسمية ومركبات الطوارئ.</p>



> Finished chain.


{'question': 'ماهي عقوبة عدم الوقوف وقوفا تاما عند الاشارة ؟',
 'context': '<h1>الغرامة المالية 5000 - 10000 ريال</h1>\n<p>قيادة المركبة تحت تأثير مسكر أو مخدر، أو عقاقير محذر من القيادة تحت تأثيرها.</p>\n\n<h1>الغرامة المالية 150 - 300 ريال</h1>\n<p>عدم ترك السائق مسافة كافية بين مركبته والمركبة التي أمامه، تسمح له بتفادي وقوع حادث.</p>\n\n<h1>الغرامة المالية 3000 - 6000 ريال</h1>\n<p>تجاوز حافلات النقل المدرسي عند توقفها للتحميل أو التنزيل.</p>\n\n<h1>الغرامة المالية 3000 - 6000 ريال</h1>\n<p>تركيب تجهيزات في المركبة كتلك الخاصة بالمركبات الرسمية ومركبات الطوارئ.</p>',
 'text': 'Based on the provided context, the answer is:\n\nالغرامة المالية 150 - 300 ريال\n\nThis is because the question asks about the penalty for not stopping completely when signaled, and the relevant penalty is listed under the heading "الغرامة المالية 150 - 300 ريال" which states "عدم ترك السائق مسافة كافية بين مركبته والمركبة التي أمامه، تسمح له بتفادي وقوع حادث."'}

In [25]:
print(f'Context:\n{response["context"]}\n\nQuestion:\n{response["question"]}\n\nText: \n{response["text"]}')

Context:
<h1>الغرامة المالية 3000 - 6000 ريال</h1>
<p>إجراء سباق للمركبات على الطرق العامة، أو السير في مواكب دون الحصول على تصريح.</p>

<h1>الغرامة المالية 150 - 300 ريال</h1>
<p>الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً.</p>

<h1>الغرامة المالية 1000 - 2000 ريال</h1>
<p>نقل عدد من الركاب يزيد على العدد المحدد في رخصة السير.</p>

<h1>الغرامة المالية 500 - 900 ريال</h1>
<p>القيادة على الطريق بمركبات مخصصة للاقتناء.</p>

Question:
ماهي الغرامة على القيادة بدون رخصة؟

Text: 
Based on the provided context, the fine for driving without a license is 3000 - 6000 ريال.
